In [1]:
import pandas as pd

In [4]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
filename_sept_train = './dataset/yellow_tripdata_2023-08.parquet'
filename_aug_validate = './dataset/yellow_tripdata_2023-09.parquet'
model_output_filepath = './model/linear_regression'

In [8]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        
        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
        
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
        
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df
        

In [9]:
df_train = read_dataframe(filename_sept_train)
df_validate = read_dataframe(filename_aug_validate)

In [10]:
len(df_train), len(df_validate)

(2742971, 2749262)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_validate['PU_DO'] = df_validate['PULocationID'] + '_' + df_validate['DOLocationID']

In [12]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_validate[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_validate[target].values

In [14]:
#Fitting the linear regression model to the training data 
lr = LinearRegression()
lr.fit(X_train, y_train)

#using the trained model to predict outcome of the validation data
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

5.9086068688901126

In [15]:
ls = Lasso(0.01)
ls.fit(X_train, y_train)

y_pred = ls.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

10.306062579741154

In [16]:
rd = Ridge(0.01)
rd.fit(X_train, y_train)

y_pred = ls.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

10.306062579741154

In [21]:
with open(f'{model_output_filepath}/lr-v-1.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)
    print("sucessfully recorded current model")

sucessfully recorded current model
